In [98]:
# Gustavo Salinas - salinasgustavon@gmail.com

In [104]:
! wget 'https://covid.ourworldindata.org/data/ecdc/full_data.csv'

--2020-04-20 19:45:05--  https://covid.ourworldindata.org/data/ecdc/full_data.csv
Resolviendo covid.ourworldindata.org (covid.ourworldindata.org)... 18.230.52.212
Conectando con covid.ourworldindata.org (covid.ourworldindata.org)[18.230.52.212]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 368552 (360K) [text/csv]
Grabando a: “full_data.csv”

full_data.csv       100%[===================>] 359.91K  1022KB/s    en 0.4s    

2020-04-20 19:45:05 (1022 KB/s) - “full_data.csv” guardado [368552/368552]



In [103]:
import os
os.unlink('full_data.csv.1')

In [101]:
#Cosas a mejorar
#   INPUT DE PAIS: elegir el país de una lista
#   FORMATO DE LOS GRAFICOS: ESTETICA PIOLA
#   SELECCION DE FECHAS ELEGIR DE UNA LISTA Y VALIDAR Q START SEA ANTES DE END


In [105]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

#Leemos el archivo y lo pasamos como diccionario
file = pd.read_csv('full_data.csv')
dataFile = file.to_dict('index')

#Guardamos todos los países en un set en mayúsculas
allCountries = set()
for key in dataFile:
    allCountries.add(dataFile[key]['location'].upper())

#Revisa que un país efectivamente exista.
def countryCheck(country):
    if country in allCountries:
        return True
    else:
        return False

#Convierte un string YYYY-mm-dd a formato date de Python
def toDate(string):
    date = datetime.datetime.strptime(string, '%Y-%m-%d')
    return date

#Plotea un diccionario
def plotDict(dict, field, format, lbl):
    x = list()
    y = list()
    for keys in dict:
        x.append(keys)
        y.append(dict[keys][field])
    plt.plot(x, y, format, label = lbl)

In [ ]:
#Primter item: la aplicación debe recibir del usuario el nombre del país deseado y graficar casos detectados y fallecimientos totales para ese país en función del tiempo.

#Identificamos el país que desea el usuario
aInput1 = str(input('Ingrese el país del que desea visualizar los contagios y fallecimientos totales:')).upper()
while countryCheck(aInput1) == False:
    aInput1= str(input('No encontramos el país que introduciste. Por favor intenta de nuevo')).upper()

#Armamos un diccionario con la info del país elegido
aCountry1 = dict()
for key in dataFile:
    if dataFile[key]['location'].upper() == aInput1:
      aCountry1[toDate(dataFile[key]['date'])] = {
          'total_cases' : int(dataFile[key]['total_cases']),
          'total_deaths' : int(dataFile[key]['total_deaths'])
          }

#Ploteamos el diccionario
plotDict(aCountry1, 'total_cases', 'k', 'Cases')
plotDict(aCountry1, 'total_deaths', 'b', 'Deaths')

plt.legend()
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Segundo item: ingresar data de dos paises y:
#   graficar cantidad de casos y fallecimiento en dos gráficos
#   el usuario indica intervalo de tiempo
#   calcular intersecciones y marcar con un punto

startDate = toDate('2020-02-20')
endDate = toDate('2020-04-18')

#Arma un diccionario con la info de cada país en el intervalo de fechas solicitado
def countryCasesDeaths(input):
    countrydict = dict()
    for key in dataFile:
        if (dataFile[key]['location'].upper() == input) and (str(startDate) <= dataFile[key]['date'] <= str(endDate)):
            countrydict[toDate(dataFile[key]['date'])] = {
            'total_cases' : int(dataFile[key]['total_cases']),
            'total_deaths' : int(dataFile[key]['total_deaths'])
             }
    return countrydict

bInput1 = 'CHINA'
bInput2 = 'ITALY'

bcountry1 = countryCasesDeaths(bInput1)
bcountry2 = countryCasesDeaths(bInput2)

#Imprimo las curvas en axes, para evitar el MatplotlibDeprecationWarning
fig, (ax1, ax2) = plt.subplots(1,2)
def plotDict1(field, format, lbl):
    x = list()
    y = list()
    for keys in bcountry1:
        x.append(keys)
        y.append(bcountry1[keys][field])
    ax1.plot(x, y, format, label = lbl)
plotDict1('total_cases','k', 'Cases')
plotDict1('total_deaths', 'b', 'Deaths')
ax1.set_title(bInput1)
ax1.set_xlabel('date')
def plotDict2(field, format, lbl):
    x = list()
    y = list()
    for keys in bcountry2:
        x.append(keys)
        y.append(bcountry2[keys][field])
    ax2.plot(x, y, format, label = lbl)
plotDict2('total_cases','k', 'Cases')
plotDict2('total_deaths', 'b', 'Deaths')
ax2.set_title(bInput2)
ax2.set_xlabel('date')


def findJoint(country1, country2, field):
    #Armo diccionario con el país que tiene el índice más alto cada día
    leadcountry = dict()
    j = 0
    for i in country1:
        if country1[i][field] > country2[i][field]:
            leadcountry[j] = {'country' : 'country1', 'date' : i}
        elif country1[i][field] < country2[i][field]:
            leadcountry[j] = {'country' : 'country2', 'date' : i}
        else:
            leadcountry[j] = {'country' : 'tie', 'date' : i}
        j += 1
    #Armo diccionario con los puntos que coinciden
    joints = dict()
    for i in range(len(leadcountry)-1):
        if (leadcountry[i]['country'] != leadcountry[i+1]['country']) or (leadcountry[i]['country'] == 'tie'):
            joints[i] = {
                'date' : leadcountry[i]['date'],
                'value': country1[leadcountry[i]['date']][field]
            }
    #Imprime los puntos
    x = list()
    y = list()
    for i in joints:
        x.append(joints[i]['date'])
        y.append(joints[i]['value'])
        ax1.plot(x,y, 'ro')
        ax2.plot(x,y, 'ro')


findJoint(bcountry1,bcountry2, 'total_cases')
findJoint(bcountry1,bcountry2, 'total_deaths')

plt.show()

In [118]:
#ITEM 4 ALMACENAR EN UN EXCEL LOS PAISES ORDENADOS DE MAYOR CANTIDAD DE CASOS A MENOR CANTIDAD DE CASOS
# INCLUIR TOTAL DE CASOS Y TOTAL DE MUERTOS
# HACER UNA HOJA POR DIA
# RANGO DE FECHAS VARIABLE

startDate = toDate('2020-02-20')
endDate = toDate('2020-04-18')

writer = pd.ExcelWriter('Resumen Diario.xlsx')

#recorro los dias uno por uno empezando por la fecha de inicio
readDate = startDate
while readDate <= endDate:
    #Creo un diccionario con las muestras de ese día
    today = dict()
    for key in dataFile:
        if readDate == toDate(dataFile[key]['date']):
            today[dataFile[key]['location']] = {
                'total_cases' : int(dataFile[key]['total_cases']),
                'total_deaths': int(dataFile[key]['total_deaths'])
            }
    #En una lista guardo la cantidad de casos del dia
    cases = list()
    for key in today:
        cases.append(today[key]['total_cases'])
    #Ordeno la lista
    cases.sort(reverse = True)
    #Creo un nuevo diccionario con el ingreso de elementos ordenado según el total de casos
    finalDict = dict()
    for i in range(len(cases)):
        for key in today:
            blacklist = list()
            if (today[key]['total_cases'] == cases[i]) and (key not in blacklist):
                finalDict[key] = {
                    'total_cases' : today[key]['total_cases'],
                    'total_deaths': today[key]['total_deaths']
                }
                #filtro para la próxima iteración el país que acabo de guardar
                blacklist.append(key)

    finalDataFrame = pd.DataFrame(finalDict)
    finalDataFrame.transpose().to_excel(writer, datetime.datetime.strftime(readDate, '%d %m %Y'))
    writer.save()
    #incremento el día y repito el bucle
    readDate += datetime.timedelta(days=1)


